In [1]:
# !pip install torch numpy pillow scikit-image pytorch-ignite ipykernel matplotlib pandas tqdm pytorch-msssim

In [2]:
import os
import time
import numpy as np
from tqdm import tqdm

print(f"Importing Pandas...")
begin = time.time()
import pandas as pd
print(f"Pandas import time: {time.time()-begin:.4f} s")

print(f"Importing Pillow...")
begin = time.time()
from PIL import Image
print(f"Pillow import time: {time.time()-begin:.4f} s")

print(f"Importing Scikit-Image...")
begin = time.time()
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
print(f"Scikit-Image import time: {time.time()-begin:.4f} s")

print(f"Importing PyTorch...")
begin = time.time()
from pytorch_msssim import ssim, ms_ssim
import torch
print(f"PyTorch import time: {time.time()-begin:.4f} s")

print(f"Importing Ignite...")
begin = time.time()
from ignite.metrics import SSIM, PSNR
print(f"Ignite import time: {time.time()-begin:.4f} s")

Importing Pandas...
Pandas import time: 2.8471 s
Importing Pillow...
Pillow import time: 0.1461 s
Importing Scikit-Image...
Scikit-Image import time: 3.9811 s
Importing PyTorch...
PyTorch import time: 16.3698 s
Importing Ignite...
Ignite import time: 10.7541 s


In [3]:
# x = 0
# for i in tqdm(range(10_000_000)):
#     x += i
# print(x)

In [4]:
DEVICE = 'cuda:0'
torch.set_default_device(DEVICE)

In [5]:
def ssim_skimage(clean_image, noisy_image):
    return structural_similarity(
        clean_image, noisy_image, win_size=11, multichannel=True,
        sigma=1.5, data_range=255, channel_axis=2, 
        use_sample_covariance=False, 
        # use_sample_covariance=True, 
        gaussian_weights=True,
        # gaussian_weights=False,
        K1=0.01, K2=0.03
    )
    
def ssim_torch(clean_image, noisy_image):
    return ssim(clean_image, noisy_image,
                data_range=255, win_size=11, 
                win_sigma=1.5, win=None, K=(0.01, 0.03),
                nonnegative_ssim=True)

ignite_ssim_cpu = SSIM(
    data_range=255, gaussian=True, sigma=(1.5, 1.5), 
    kernel_size=(11, 11), 
    k1=0.01, k2=0.03,
    device="cpu")

ignite_ssim_cuda = SSIM(
    data_range=255, gaussian=True, sigma=(1.5, 1.5), 
    kernel_size=(11, 11), 
    k1=0.01, k2=0.03,
    device=DEVICE)

def ssim_ignite_cpu(clean_image, noisy_image):
    ignite_ssim_cpu.reset()
    ignite_ssim_cpu.update((clean_image, noisy_image))
    return ignite_ssim_cpu.compute()

def ssim_ignite_cuda(clean_image, noisy_image):
    ignite_ssim_cuda.reset()
    ignite_ssim_cuda.update((clean_image, noisy_image))
    return ignite_ssim_cuda.compute()

In [6]:
def psnr_skimage(clean_image, noisy_image):
    return peak_signal_noise_ratio(clean_image, noisy_image, data_range=255)

ignite_psnr_cpu = PSNR(data_range=255, device="cpu")

ignite_psnr_cuda = PSNR(data_range=255, device=DEVICE)

def psnr_ignite_cpu(clean_image, noisy_image):
    ignite_psnr_cpu.reset()
    ignite_psnr_cpu.update((clean_image, noisy_image))
    return ignite_psnr_cpu.compute()

def psnr_ignite_cuda(clean_image, noisy_image):
    ignite_psnr_cuda.reset()
    ignite_psnr_cuda.update((clean_image, noisy_image))
    return ignite_psnr_cuda.compute()

In [7]:
sigma_range = 101
sigmas = list(range(0, sigma_range, 10))

In [8]:
df_result = pd.DataFrame(columns=[
    'sigma', 'ski_ssim', 
    'pm_ssim_cpu', 'pm_ssim_cuda', 
    'ignite_cpu', 'ignite_cuda', 
    'pm_msssim_cpu', 'pm_msssim_cuda',
    'ski_psnr', 'ignite_psnr_cpu', 'ignite_psnr_cuda',
])
df_time = pd.DataFrame(columns=[
    'sigma', 'ski_ssim (ms)',
    'pm_ssim_cpu (ms)', 'pm_ssim_cuda (ms)',
    'ignite_cpu (ms)', 'ignite_cuda (ms)',
    'pm_msssim_cpu (ms)', 'pm_msssim_cuda (ms)',
    'ski_psnr (ms)', 'ignite_psnr_cpu (ms)', 'ignite_psnr_cuda (ms)',
])
# Set sigmas in df
df_result['sigma'] = sigmas
df_time['sigma'] = sigmas

In [9]:
import urllib
test_image = 'kodim10.png'
if not os.path.isfile("kodim10.png"):
    print("Downloading test image kodim10.png...")
    urllib.request.urlretrieve(
        "http://r0k.us/graphics/kodak/kodak/kodim10.png", "kodim10.png")

img = Image.open('kodim10.png')
img = np.array(img).astype(np.float32)

# params = torch.nn.Parameter( torch.ones(img.shape[2], img.shape[0], img.shape[1]), requires_grad=True ) # C, H, W
img_torch_cpu = torch.from_numpy(img).to(device="cpu").unsqueeze(0).permute(0, 3, 1, 2)  # 1, C, H, W
img_torch_cuda = img_torch_cpu.to(device=DEVICE)

In [10]:
img_batch_cpu = []
img_noise_batch_cpu = []
single_image_ssim_cpu = []

img_batch_cuda = []
img_noise_batch_cuda = []
single_image_ssim_cuda = []

noisy_images_np = []
noisy_images_torch_cpu = []
noisy_images_torch_cuda = []

np.random.seed(42)

for sigma in sigmas:
    noise = sigma * np.random.rand(*img.shape)
    img_noise = (img + noise).astype(np.float32).clip(0,255)
    noisy_images_np.append(img_noise)
    img_noise_torch_cpu = torch.from_numpy(img_noise).to(device="cpu").unsqueeze(0).permute(0, 3, 1, 2)  # 1, C, H, W
    img_noise_torch_cuda = img_noise_torch_cpu.to(device=DEVICE)
    noisy_images_torch_cpu.append(img_noise_torch_cpu)
    noisy_images_torch_cuda.append(img_noise_torch_cuda)
    
    img_batch_cpu.append(img_torch_cpu)
    img_noise_batch_cpu.append(img_noise_torch_cpu)
    
    img_batch_cuda.append(img_torch_cuda)
    img_noise_batch_cuda.append(img_noise_torch_cuda)

In [11]:
def process(clean_image, noisy_images, func, col, n_repeat=1):
    for i in tqdm(range(len(sigmas))):
        sigma = sigmas[i]
        img_noise = noisy_images[i]
        
        begin = time.time()
        for _ in range(n_repeat):
            score = func(clean_image, img_noise)
        time_ms = (time.time()-begin) / n_repeat * 1000
        
        # Convert to numpy if tensor
        if isinstance(score, torch.Tensor):
            score = score.cpu().numpy()
        df_result.loc[df_result['sigma']==sigma, col] = score
        df_time.loc[df_time['sigma']==sigma, f"{col} (ms)"] = time_ms

In [12]:
# Warm up the GPU
process(img_torch_cuda, noisy_images_torch_cuda, ssim_torch, 'pm_ssim_cuda', n_repeat=10)
process(img_torch_cuda, noisy_images_torch_cuda, ssim_ignite_cuda, 'ignite_cuda', n_repeat=10)
process(img_torch_cuda, noisy_images_torch_cuda, ms_ssim, 'pm_msssim_cuda', n_repeat=10)
process(img_torch_cuda, noisy_images_torch_cuda, psnr_ignite_cuda, 'ignite_psnr_cuda', n_repeat=10)

100%|██████████| 11/11 [00:00<00:00, 51.52it/s]


In [13]:
N_REPEAT = 200

In [14]:
# Actual GPU benchmark
process(img_torch_cuda, noisy_images_torch_cuda, ssim_torch, 'pm_ssim_cuda', n_repeat=N_REPEAT)
process(img_torch_cuda, noisy_images_torch_cuda, ssim_ignite_cuda, 'ignite_cuda', n_repeat=N_REPEAT)
process(img_torch_cuda, noisy_images_torch_cuda, ms_ssim, 'pm_msssim_cuda', n_repeat=N_REPEAT)
process(img_torch_cuda, noisy_images_torch_cuda, psnr_ignite_cuda, 'ignite_psnr_cuda', n_repeat=N_REPEAT)

100%|██████████| 11/11 [00:01<00:00, 10.25it/s]


In [15]:
df_result

,sigma,ski_ssim,pm_ssim_cpu,pm_ssim_cuda,ignite_cpu,ignite_cuda,pm_msssim_cpu,pm_msssim_cuda,ski_psnr,ignite_psnr_cpu,ignite_psnr_cuda
0,0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,148.130804
1,10,NaN,NaN,0.925051,NaN,0.93338,NaN,0.989004,NaN,NaN,32.940875
2,20,NaN,NaN,0.773435,NaN,0.788487,NaN,0.963898,NaN,NaN,26.917335
3,30,NaN,NaN,0.62326,NaN,0.641529,NaN,0.929061,NaN,NaN,23.400935
4,40,NaN,NaN,0.502339,NaN,0.521008,NaN,0.890179,NaN,NaN,20.906226
5,50,NaN,NaN,0.410533,NaN,0.428362,NaN,0.850126,NaN,NaN,18.980003
6,60,NaN,NaN,0.340449,NaN,0.357037,NaN,0.811729,NaN,NaN,17.402383
7,70,NaN,NaN,0.286809,NaN,0.301982,NaN,0.772992,NaN,NaN,16.070275
8,80,NaN,NaN,0.245654,NaN,0.259704,NaN,0.736515,NaN,NaN,14.937702
9,90,NaN,NaN,0.212196,NaN,0.225221,NaN,0.703794,NaN,NaN,13.928136


In [16]:
df_time

,sigma,ski_ssim (ms),pm_ssim_cpu (ms),pm_ssim_cuda (ms),ignite_cpu (ms),ignite_cuda (ms),pm_msssim_cpu (ms),pm_msssim_cuda (ms),ski_psnr (ms),ignite_psnr_cpu (ms),ignite_psnr_cuda (ms)
0,0,NaN,NaN,1.750603,NaN,0.971634,NaN,5.962734,NaN,NaN,0.487914
1,10,NaN,NaN,1.776279,NaN,0.979192,NaN,6.016423,NaN,NaN,0.48537
2,20,NaN,NaN,1.768124,NaN,0.9683,NaN,5.960847,NaN,NaN,0.481173
3,30,NaN,NaN,1.76796,NaN,0.966797,NaN,5.902171,NaN,NaN,0.483071
4,40,NaN,NaN,1.766243,NaN,0.972424,NaN,5.969783,NaN,NaN,0.47866
5,50,NaN,NaN,1.769297,NaN,0.970341,NaN,6.019173,NaN,NaN,0.480751
6,60,NaN,NaN,1.768923,NaN,0.970281,NaN,6.104696,NaN,NaN,0.48169
7,70,NaN,NaN,1.768587,NaN,0.97316,NaN,6.015153,NaN,NaN,0.478048
8,80,NaN,NaN,1.771626,NaN,0.963473,NaN,6.045595,NaN,NaN,0.485873
9,90,NaN,NaN,1.763848,NaN,0.968945,NaN,6.012934,NaN,NaN,0.486146


In [17]:
process(img, noisy_images_np, ssim_skimage, 'ski_ssim', n_repeat=N_REPEAT)

100%|██████████| 11/11 [02:36<00:00, 14.24s/it]


In [18]:
process(img_torch_cpu, noisy_images_torch_cpu, ssim_torch, 'pm_ssim_cpu', n_repeat=N_REPEAT)

100%|██████████| 11/11 [00:39<00:00,  3.57s/it]


In [19]:
process(img_torch_cpu, noisy_images_torch_cpu, ssim_ignite_cpu, 'ignite_cpu', n_repeat=N_REPEAT)

100%|██████████| 11/11 [01:07<00:00,  6.18s/it]


In [20]:
process(img_torch_cpu, noisy_images_torch_cpu, ms_ssim, 'pm_msssim_cpu', n_repeat=N_REPEAT)

100%|██████████| 11/11 [00:59<00:00,  5.37s/it]


In [21]:
process(img, noisy_images_np, psnr_skimage, 'ski_psnr', n_repeat=N_REPEAT)

  0%|          | 0/11 [00:00<?, ?it/s]/mnt/h/GIT/DISSERTATION/ssim-implementations-comparison/venv/lib/python3.10/site-packages/skimage/metrics/simple_metrics.py:168: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range**2) / err)
100%|██████████| 11/11 [00:01<00:00,  9.25it/s]


In [22]:
process(img_torch_cpu, noisy_images_torch_cpu, psnr_ignite_cpu, 'ignite_psnr_cpu', n_repeat=N_REPEAT)

100%|██████████| 11/11 [00:02<00:00,  5.43it/s]


In [23]:
df_result

,sigma,ski_ssim,pm_ssim_cpu,pm_ssim_cuda,ignite_cpu,ignite_cuda,pm_msssim_cpu,pm_msssim_cuda,ski_psnr,ignite_psnr_cpu,ignite_psnr_cuda
0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,inf,148.130804,148.130804
1,10,0.932475,0.932476,0.925051,0.93338,0.93338,0.991116,0.989004,32.940875,32.940875,32.940875
2,20,0.785769,0.78577,0.773435,0.788488,0.788487,0.967452,0.963898,26.917335,26.917335,26.917335
3,30,0.637239,0.637241,0.62326,0.64153,0.641529,0.934444,0.929061,23.400935,23.400935,23.400935
4,40,0.515662,0.515664,0.502339,0.521009,0.521008,0.897658,0.890179,20.906226,20.906226,20.906226
5,50,0.422473,0.422474,0.410533,0.428363,0.428362,0.858975,0.850126,18.980003,18.980003,18.980003
6,60,0.350911,0.350912,0.340449,0.357038,0.357037,0.821748,0.811729,17.402383,17.402383,17.402383
7,70,0.295807,0.295808,0.286809,0.301982,0.301982,0.784397,0.772992,16.070275,16.070275,16.070275
8,80,0.253616,0.253617,0.245654,0.259705,0.259704,0.748474,0.736515,14.937702,14.937702,14.937702
9,90,0.219286,0.219287,0.212196,0.225221,0.225221,0.716415,0.703794,13.928136,13.928136,13.928136


In [24]:
df_time

,sigma,ski_ssim (ms),pm_ssim_cpu (ms),pm_ssim_cuda (ms),ignite_cpu (ms),ignite_cuda (ms),pm_msssim_cpu (ms),pm_msssim_cuda (ms),ski_psnr (ms),ignite_psnr_cpu (ms),ignite_psnr_cuda (ms)
0,0,69.681405,18.10609,1.750603,30.51495,0.971634,25.284748,5.962734,0.57246,1.175711,0.487914
1,10,69.783146,17.789878,1.776279,27.012008,0.979192,27.424959,6.016423,0.529858,0.904857,0.48537
2,20,71.580648,17.924452,1.768124,30.750805,0.9683,27.864925,5.960847,0.532987,0.911472,0.481173
3,30,71.022345,19.35635,1.76796,30.879649,0.966797,27.689744,5.902171,0.523221,0.919428,0.483071
4,40,69.945178,17.440642,1.766243,29.888916,0.972424,26.870123,5.969783,0.524347,0.917566,0.47866
5,50,71.5877,18.210635,1.769297,28.137956,0.970341,27.394352,6.019173,0.529437,0.9038,0.480751
6,60,71.114575,17.928377,1.768923,30.286449,0.970281,25.160593,6.104696,0.52575,0.95239,0.48169
7,70,71.144938,17.758013,1.768587,33.776784,0.97316,26.843193,6.015153,0.524337,0.866969,0.478048
8,80,72.137105,17.378492,1.771626,31.016955,0.963473,27.380863,6.045595,0.519882,0.75506,0.485873
9,90,72.42509,16.859591,1.763848,33.415898,0.968945,27.742596,6.012934,0.542189,0.931644,0.486146
